# Machine Learning In Production

<img src='https://imgur.com/orZWHly.png' alt='Penguins dataset' width="900">

This notebook was created by [Santiago L. Valdarrama](https://twitter.com/svpino) as part of the [Machine Learning School](https://www.ml.school) program.

In [147]:
%load_ext autoreload
%autoreload 2

In [5]:
# Let's make sure we are running the latest version of the SakeMaker's SDK. 
# Restart the notebook after you upgrade the library.

!pip install -q --upgrade pip
!pip install -q --upgrade sagemaker
!pip show sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.22.22 requires botocore==1.23.22, but you have botocore 1.29.114 which is incompatible.
awscli 1.22.22 requires s3transfer<0.6.0,>=0.5.0, but you have s3transfer 0.6.0 which is incompatible.
Name: sagemaker
Version: 2.146.0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.8/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, jsonschema, numpy, packaging, pandas, pathos, platformdirs, protobuf, protobuf3-to-dict, PyYAML, schema, smdebug-rulesconfig
Required-by: 


If you set the following constant to a specific pipeline, you can run the entire notebook but only the specified pipeline will run.

Here are the possible values:

* PREPROCESSING
* TRAINING
* TUNING
* EVALUATION
* DEPLOYMENT
* CUSTOM_ENDPOINT
* MONITORING

For example, setting `PIPELINE` to `DEPLOYMENT` will only execute the deployment pipeline from Session 4.

In [2]:
PIPELINE = "CUSTOM_ENDPOINT"

# Session 1 - Getting Started

This session aims to build a simple [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with one step to preprocess the dataset.


In [3]:
import os
import sagemaker
import numpy as np
import boto3
import json
import pandas as pd
import numpy as np
import urllib.request
import argparse
import tempfile
from pathlib import Path

from botocore.exceptions import ClientError
from sagemaker.inputs import FileSystemInput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.processing import ScriptProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterInteger, ParameterString, ParameterFloat
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CacheConfig


role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()

## Step 1 - Creating an S3 Bucket

We need to create an S3 bucket where we will upload everything we need during the program.

Make sure you set `BUCKET` to the name of the bucket you want to use.

In [4]:
BUCKET = "mlschool"

!aws s3api create-bucket --bucket $BUCKET

{
    "Location": "/mlschool"
}


## Step 2 - Downloading the Dataset

We can now download the [Penguins dataset](https://www.kaggle.com/parulpandey/palmer-archipelago-antarctica-penguin-data) and store it in S3.

In [5]:
S3_FILEPATH = f"s3://{BUCKET}/penguins"
DATA_FILEPATH = "penguins/data.csv"

# Download the official Penguins dataset and store it locally.
urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

# Upload the dataset to S3. We need to do this to make it available to 
# the preprocessing step.
INPUT_DATA_URI = sagemaker.s3.S3Uploader.upload(
    local_path=DATA_FILEPATH, 
    desired_s3_uri=S3_FILEPATH,
)

print(f"Dataset S3 location: {INPUT_DATA_URI}")

Dataset S3 location: s3://mlschool/penguins/data.csv


We can now load and display the dataset.

In [6]:
df = pd.read_csv(DATA_FILEPATH)
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


## Step 3 - Preprocessing the Dataset

Let's create a script to do feature engineering on the original dataset. 

This script should also split the data into train, validation, and a test set so we can later train and evaluate a model. We will save the Scikit-Learn pipeline that we use to preprocess the data to use it during inference time.

In [7]:
%%writefile penguins/preprocessor.py

import os
import numpy as np
import pandas as pd
import tempfile

from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from pickle import dump


BASE_DIR = "/opt/ml/processing"
DATA_FILEPATH = Path(BASE_DIR) / "input" / "data.csv"


def save_splits(base_dir, train, validation, test):
    """
    Saves the supplied datasets to disk.
    """
    
    train_path = Path(base_dir) / "train" 
    validation_path = Path(base_dir) / "validation" 
    test_path = Path(base_dir) / "test"
    
    train_path.mkdir(parents=True, exist_ok=True)
    validation_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)
    
    pd.DataFrame(train).to_csv(train_path / "train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(validation_path / "validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(test_path / "test.csv", header=False, index=False)

    
def save_pipeline(base_dir, pipeline):
    """
    Saves the Scikit-Learn pipeline that we used to
    preprocess the data.
    """
    pipeline_path = Path(base_dir) / "pipeline"
    pipeline_path.mkdir(parents=True, exist_ok=True)
    dump(pipeline, open(pipeline_path / "pipeline.pkl", 'wb'))
    

def preprocess(base_dir, data_filepath):
    """
    Preprocesses the supplied raw dataset and splits it into a train, validation,
    and a test set.
    """
    
    df = pd.read_csv(data_filepath)
    
    numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]
    
    numerical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    categorical_preprocessor = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("numerical", numerical_preprocessor, numerical_columns),
            ("categorical", categorical_preprocessor, ["island"]),
        ]
    )
    

    X = df.drop(["sex"], axis=1)
    columns = list(X.columns)
    
    X = X.to_numpy()
    
    np.random.shuffle(X)
    train, validation, test = np.split(X, [int(.7 * len(X)), int(.85 * len(X))])
    
    X_train = pd.DataFrame(train, columns=columns)
    X_validation = pd.DataFrame(validation, columns=columns)
    X_test = pd.DataFrame(test, columns=columns)
    
    y_train = X_train.species
    y_validation = X_validation.species
    y_test = X_test.species

    X_train.drop(["species"], axis=1, inplace=True)
    X_validation.drop(["species"], axis=1, inplace=True)
    X_test.drop(["species"], axis=1, inplace=True)

    X_train = preprocessor.fit_transform(X_train)
    X_validation = preprocessor.transform(X_validation)
    X_test = preprocessor.transform(X_test)

    label_encoder = LabelEncoder()
    
    y_train = label_encoder.fit_transform(y_train)
    y_validation = label_encoder.transform(y_validation)
    y_test = label_encoder.transform(y_test)
    
    
    train = np.concatenate((X_train, np.expand_dims(y_train, axis=1)), axis=1)
    validation = np.concatenate((X_validation, np.expand_dims(y_validation, axis=1)), axis=1)
    test = np.concatenate((X_test, np.expand_dims(y_test, axis=1)), axis=1)
    
    save_splits(base_dir, train, validation, test)
    save_pipeline(base_dir, pipeline=preprocessor)
        

if __name__ == "__main__":
    preprocess(BASE_DIR, DATA_FILEPATH)


Overwriting penguins/preprocessor.py


## Step 4 - Testing the Preprocessing Script

We can now load the script we just created and run it locally to ensure it creates the 3 splits. 

Having a way to run scripts locally is crucial to shorten the development feedback.

In [8]:
from penguins.preprocessor import preprocess

with tempfile.TemporaryDirectory() as directory:
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )
    
    print(f"Splits: {os.listdir(directory)}")
    print(f"Train: {os.listdir(Path(directory) / 'train')}")
    print(f"Validation: {os.listdir(Path(directory) / 'validation')}")
    print(f"Test: {os.listdir(Path(directory) / 'test')}")
    print(f"Pipeline: {os.listdir(Path(directory) / 'pipeline')}")

Splits: ['train', 'validation', 'test', 'pipeline']
Train: ['train.csv']
Validation: ['validation.csv']
Test: ['test.csv']
Pipeline: ['pipeline.pkl']


## Step 5 - Pipeline Configuration

When we create a SageMaker Pipeline we can specify a list of paramaters that we can use throughout the individual pipeline steps. To read more about these parameters, check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html).

These are the parameters that will use in our pipeline:

* `dataset_location`: This parameter represents the location of the dataset in S3. We will use this parameter during the preprocessing step to access the dataset.
* `preprocessor_destination`: We need to define the location where the preprocessing step will be storing the dataset splits and the preprocessing pipeline to avoid SageMaker from appending a timestamp to their auto-generated location. If we let SageMaker use a timestamp, we can't cache this step.

In [9]:
dataset_location = ParameterString(
    name="dataset_location",
    default_value=INPUT_DATA_URI,
)

preprocessor_destination = ParameterString(
    name="preprocessor_destination",
    default_value=f'{S3_FILEPATH}/preprocessing',
)

## Step 6 - Caching Pipeline Steps

While you are building your pipeline, you don't want to rerun every step of the process unless you expect a different result. Instead, you can instruct SageMaker to reuse the result of a previous successful run of a pipeline step.

You can accomplish this by caching your steps. You can find more information about this topic in [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html).

Getting caching to work is tricky, and you will find SageMaker missing the cache frequently. Whenever that happens, you need to dig and figure out how to adjust the step configuration to prevent SageMaker from autogenerating data that prevents a cache hit. For example, to cache the preprocessing step we need to define the destination of the processing job to prevent SageMaker from using an autogenerated timestamp.

In [10]:
# We'll use this cache configuration to cache individual steps for 
# a maximum of 15 days.
cache_config = CacheConfig(
    enable_caching=True, 
    expire_after="15d"
)

## Step 7 - Setting up a Processing Step

The first step we need in our pipeline is a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) to run the preprocessing script. Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.

To run our script, we need access to Scikit-Learn, so we can use the [SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-processor) processor that comes out-of-the-box with the SageMaker's Python SDK.

The input of this step will be the dataset location. The outputs will be the three sets and the Scikit-Learn preprocessing pipeline.

In [11]:
sklearn_processor = SKLearnProcessor(
    base_job_name="penguins-preprocessing",
    framework_version="0.23-1",
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

preprocess_step = ProcessingStep(
    name="preprocessing",
    processor=sklearn_processor,
    inputs=[
        ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=preprocessor_destination),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=preprocessor_destination),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=preprocessor_destination),
        ProcessingOutput(output_name="pipeline", source="/opt/ml/processing/pipeline", destination=preprocessor_destination),
    ],
    code="penguins/preprocessor.py",
    cache_config=cache_config
)

## Step 8 - Defining and Running the Pipeline

We can now define and run the SageMaker Pipeline. Check [Pipeline Structure and Execution](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-pipeline.html) for more information about how to define a pipeline and [Run a Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/run-pipeline.html) for information about how to run it.


In [12]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [13]:
if PIPELINE == "PREPROCESSING":
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()

## Step 9 - Cleaning up

Before you finish, don't forget to clean up after you.

In [181]:
session1_pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session1-penguins-pipeline',
 'ResponseMetadata': {'RequestId': '32f6ae54-7f2f-4fba-a7a1-a7c4c276624a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '32f6ae54-7f2f-4fba-a7a1-a7c4c276624a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '94',
   'date': 'Wed, 12 Apr 2023 23:02:57 GMT'},
  'RetryAttempts': 0}}

## Assignments

1. Set up an Amazon SageMaker domain using the Standard Setup. Make sure you set the network configuration to VPC Only. Create a new execution role and ensure it has access to the S3 bucket you’ll use during this class. You can also specify “Any S3 bucket” if you want this role to access every S3 bucket in your AWS account.

2. Create a GitHub repository and clone it from inside SageMaker Studio. We’ll use this repository to store the code used during this program.

3. Configure your SageMaker Studio session to store your name and email address and cache your credentials. You can use the following commands from a Terminal window:

```bash
$ git config --global user.name "John Doe"
$ git config --global user.email johndoe@example.com
$ git config --global credential.helper store
```

4. Throughout the course, you will work on the "Pipeline of Digits" project with the goal of seting up a SageMaker pipeline for a simple computer vision project. For this assignment, open the `mnist.ipynb` notebook and follow the instructions to prepare the dataset for the project.

5. Setup a SageMaker pipeline for the "Pipeline of Digits" project. Create a preprocessing step where you split the MNIST dataset into a train and a test set.

## References

1. Amazon SageMaker is free to try. Your free tier starts from the first month when you create your first SageMaker resource and lasts 2 months. Check out the [Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/) for more information.

2. We’ll be working extensively with [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) and [SageMaker’s Python SDK](https://sagemaker.readthedocs.io/en/stable/). Keep their documentation handy.

3. Check the [SageMaker Pipelines Overview](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) for an introduction to the fundamental components of a SageMaker Pipeline.

4. Check [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) for more information on how to define and use variables in your pipeline.

5. Check [Caching Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-caching.html) for information on how to cache the results of individual pipeline steps.

6. Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation. You can find an example of how to create a processing job from the pipeline in the [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing) page.

7. Check [Pipeline Structure and Execution](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-pipeline.html) for more information about how to define a pipeline.

8. Check [Run a Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/run-pipeline.html) for information about how to submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does, and then run it.

## Additional Notes

1. This notebook uses a Scikit-Learn Pipeline to transform the dataset. You should always orchestrate your transformations using pipelines. Check the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) for more details.

2. The preprocessing script uses `np.split()` to split the dataset into 3 different splits. It's a neat way of getting the three splits with a single instruction.

3. Keras offers a [list of built-in vectorized datasets](https://www.notion.so/Bnomial-RESTful-API-4ecf85043b484ec994d7f70c56abfe27) in NumPy format. You can load any of these datasets with a single line of code, making them convenient.

4. Converting a Numpy array into an image you can save and visualize is a useful trick to know. Check the `Image.fromarray()` function from the `PIL` library.

5. The [command line interface](https://docs.aws.amazon.com/cli/latest/index.html) is a simple way to interact with the AWS services. You can combine Python code with bash commands in the same notebook cell, which makes notebooks a very flexible tool.

6. Check Python’s `pathlib` module. Since Python 3.4, this module offers a clean way to interact with the filesystem.

7. This notebook uses the `%%writefile`, `%load_ext`, and `%autoreload` magics. These magics are very useful when using notebooks. Check this list of [line and cell magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html) for other examples.

# Session 2 - Training and Tuning

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) we built in the previous session with one step that trains a model.

We explore the [Training](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) and the [Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) steps.


In [14]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TuningStep
from sagemaker.parameter import IntegerParameter
from sagemaker.inputs import TrainingInput
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline_context import PipelineSession

## Step 1 - Training the Model

This script is responsible from training a simple neural network on the train data, validating the model, and saving it so we can later use it.

In [15]:
%%writefile penguins/train.py

import os
import argparse

import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD


def train(base_directory, train_path, validation_path, epochs=50, batch_size=32):
    X_train = pd.read_csv(Path(train_path) / "train.csv")
    y_train = X_train[X_train.columns[-1]]
    X_train.drop(X_train.columns[-1], axis=1, inplace=True)
    
    X_validation = pd.read_csv(Path(validation_path) / "validation.csv")
    y_validation = X_validation[X_validation.columns[-1]]
    X_validation.drop(X_validation.columns[-1], axis=1, inplace=True)
    
    model = Sequential([
        Dense(10, input_shape=(X_train.shape[1],), activation="relu"),
        Dense(8, activation="relu"),
        Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=SGD(learning_rate=0.01),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.fit(
        X_train, 
        y_train, 
        validation_data=(X_validation, y_validation),
        epochs=epochs, 
        batch_size=batch_size,
        verbose=2,
    )

    predictions = np.argmax(model.predict(X_validation), axis=-1)
    print(f"Validation accuracy: {accuracy_score(y_validation, predictions)}")
    
    model_filepath = Path(base_directory) / "model" / "001"
    model.save(model_filepath)
    
if __name__ == "__main__":
    # Any hyperparameters provided by the training job are passed to the entry point
    # as script arguments. SageMaker will also provide a list of special parameters
    # that you can capture here. Here is the full list: 
    # https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py
    parser = argparse.ArgumentParser()
    parser.add_argument("--base_directory", type=str, default="/opt/ml/")
    parser.add_argument("--train_path", type=str, default=os.environ.get("SM_CHANNEL_TRAIN", None))
    parser.add_argument("--validation_path", type=str, default=os.environ.get("SM_CHANNEL_VALIDATION", None))
    parser.add_argument("--epochs", type=int, default=50)
    parser.add_argument("--batch_size", type=int, default=32)
    args, _ = parser.parse_known_args()
    
    train(
        base_directory=args.base_directory,
        train_path=args.train_path,
        validation_path=args.validation_path,
        epochs=args.epochs,
        batch_size=args.batch_size
    )

Overwriting penguins/train.py


## Step 2 - Testing the Training Script

Let's test the script we just created by running it locally.

In [22]:
from penguins.preprocessor import preprocess
from penguins.train import train


with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )

    # Then, we train a model using the train and 
    # validation splits.
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=10
    )

Epoch 1/50
Extension horovod.torch has not been built: /usr/local/lib/python3.8/site-packages/horovod/torch/mpi_lib/_mpi_lib.cpython-38-x86_64-linux-gnu.so not found
If this is not expected, reinstall Horovod with HOROVOD_WITH_PYTORCH=1 to debug the build error.
Warning! MPI libs are missing, but python applications are still avaiable.
[2023-04-13 20:23:54.266 tensorflow-2-6-cpu-py-ml-t3-medium-9169b2e75617c45c79c40579f6a8:105 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-04-13 20:23:54.332 tensorflow-2-6-cpu-py-ml-t3-medium-9169b2e75617c45c79c40579f6a8:105 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
8/8 - 1s - loss: 1.0914 - accuracy: 0.2720 - val_loss: 1.0718 - val_accuracy: 0.3137
Epoch 2/50
8/8 - 0s - loss: 1.0455 - accuracy: 0.3975 - val_loss: 1.0402 - val_accuracy: 0.3529
Epoch 3/50
8/8 - 0s - loss: 1.0129 - accuracy: 0.4435 - val_loss: 1.0136 - val_accuracy: 0.3137
Epoch 4/50
8/8 - 0s 

INFO:tensorflow:Assets written to: /tmp/tmp10_u1g1g/model/001/assets


## Step 3 - Configuring an Estimator

SageMaker uses the concept of an [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to handle end-to-end training and deployment tasks. For this example, we will use the [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) to run the trainning script we wrote before.

SageMaker will pass the list of hyperparameters defined below to the entry point of the training script as arguments.

In [16]:
hyperparameters = {
    "epochs": 50,
    "batch_size": 32
}

estimator = TensorFlow(
    entry_point="penguins/train.py",
    role=role,
    hyperparameters=hyperparameters,
    instance_type="ml.m5.large",
    instance_count=1,
    py_version="py37",
    framework_version="2.4",
    script_mode=True,
    disable_profiler=True
)

## Step 4 - Setting up a Training Step

We can now create a [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) that we can add to the pipeline. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation for more information. 

This step will use the estimator we configured before and will receive the train and validation splits from the preprocessing step as inputs.

In [17]:
training_step = TrainingStep(
    name="training",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

## Step 5 - Running the Pipeline with the Training Step

We can now define and run the SageMaker Pipeline, this time using the new Training Step.

In [18]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        training_step
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [19]:
if PIPELINE == "TRAINING":
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()

## Step 6 - Configuring a Hyperparameter Tuner

An alternative to training a model is to train many variants of the model and choose the best one. We can do this with an instance of the [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) class.

The tuner uses the same `Estimator` we defined to train the model, and we can specify how it should determine the best model:

1. `objective_metric_name`: This is the name of the metric the tuner will use to determine the best model.
2. `objective_type`: This is the objective of the tuner. Should it "Minimize" the metric or "Maximize" it? In this example, sice we are using the validation accuracy of the model, we want the objetive to be "Maximize." If we were using the loss of the model, we would set the objetive to "Minimize."
3. `metric_definitions`: Defines how the tuner will determine the value of the metric by looking at the output logs of the training process.

The tuner expects a list of the hyperparameters you want to explore. You can use subclasses of the [Parameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ParameterRange) class to specify different types of hyperparameters.

Finally, you can control the number of jobs and how many of them will run in parallel using the following two arguments:
* `max_jobs`: Defines the maximum total number of training jobs to start for the hyperparameter tuning job.
* `max_parallel_jobs`: Defines the maximum number of parallel training jobs to start.

In [20]:
hyperparameter_ranges = {
    "epochs": IntegerParameter(10, 50)
}

objective_metric_name = "val_accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": objective_metric_name, "Regex": "val_accuracy: ([0-9\\.]+)"}]
    
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type=objective_type,
    max_jobs=3,
    max_parallel_jobs=3,
)

## Step 7 - Setting up a Tuning Step

We can now create a [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) to add it to our pipeline. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation for more information. 

This step will use the tuner we configured before and will receive the train and validation splits from the preprocessing step as inputs.

In [21]:
tuning_step = TuningStep(
    name = "tuning",
    tuner=tuner,
    inputs={
        "train": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
    cache_config=cache_config
)

## Step 8 - Running the Pipeline with the Tuning Step

We can now define and run the SageMaker Pipeline, this time using the new Tuning Step.

In [22]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
    ],
    steps=[
        preprocess_step, 
        tuning_step
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [23]:
if PIPELINE == "TUNING":
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()

## Step 9 - Cleaning up

Before you finish, don't forget to clean up after you.

In [29]:
session2_training_pipeline.delete()
session2_tuning_pipeline.delete()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:325223348818:pipeline/session2-tuning-penguins-pipeline',
 'ResponseMetadata': {'RequestId': 'fb4b7e41-1fb4-4077-887b-15ca1bf09910',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fb4b7e41-1fb4-4077-887b-15ca1bf09910',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '101',
   'date': 'Tue, 11 Apr 2023 20:35:46 GMT'},
  'RetryAttempts': 2}}

## Assignments

1. Modify the training script so it accepts the `learning_rate` as a new hyperparameter using the list of hyperparameters supplied to the Estimator.

2. Replace the TensorFlow Estimator with a Pytorch Estimator. Check [this page](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#create-an-estimator) for an example of how to create a PyTorch Estimator. You'll need to create a new training script that builds a PyTorch model to solve the problem.

3. Modify the tuner job to find the best `learning_rate` value between `0.01` and `0.03`. Check the [ContinuousParameter](https://sagemaker.readthedocs.io/en/stable/api/training/parameter.html#sagemaker.parameter.ContinuousParameter) class for more information on how to configure this parameter.

4. Modify the pipeline to run the training and tuning jobs concurrently.

5. Modify the SageMaker Pipeline you created for the MNIST project and add a training step. That step should only receive one channel with the train data. 

## References

1. The [Docker Registry Paths and Example Code](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html) page contains information about the available fremwork versions for each region. You can also check the available SageMaker [Deep Learning Container images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) here.

2. Check [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit) for more information about how to train machine learning models within a Docker container using Amazon SageMaker.

3. Check the [TrainingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TrainingStep) SageMaker's SDK documentation. You can find an example of how to create a training job from the pipeline in the [Training Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-training) page.

4. Check the [TuningStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep) SageMaker's SDK documentation. You can find an example of how to create a hyperparameter tuning job from the pipeline in the [Tuning Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-tuning) page.

5. Check the [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) and the [TensorFlow Estimator](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator) documentation for more information about how these classes work. You can also find [other supported frameworks](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html) in the documentation.

6. Check the [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) class for more information about how to configure a hyperparameter job.

7. The SageMaker SDK passes special hyperparameters to the training job that we can capture from inside the script. Here is the [complete list of available hyperparameters](https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/params.py). 



# Session 3 - Evaluating the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to evaluate the model.


In [24]:
import tarfile

from sagemaker.workflow.properties import PropertyFile

## Step 1 - Evaluating the Model

This script is reponsible from loading the model we created and evaluating it on the test set. Before finishing, this script will create a file containing an evaluation report of the model.

In [25]:
%%writefile penguins/evaluation.py

import os
import json
import tarfile
import numpy as np
import pandas as pd

from pathlib import Path
from tensorflow import keras
from sklearn.metrics import accuracy_score


MODEL_PATH = "/opt/ml/processing/model/"
TEST_PATH = "/opt/ml/processing/test/"
OUTPUT_PATH = "/opt/ml/processing/evaluation/"


def evaluate(model_path, test_path, output_path):
    # The first step is to extract the model package provided
    # by SageMaker.
    with tarfile.open(Path(model_path) / "model.tar.gz") as tar:
        tar.extractall(path=Path(model_path))
        
    # We can now load the model from disk.
    model = keras.models.load_model(Path(model_path) / "001")
    
    X_test = pd.read_csv(Path(test_path) / "test.csv")
    y_test = X_test[X_test.columns[-1]]
    X_test.drop(X_test.columns[-1], axis=1, inplace=True)
    
    predictions = np.argmax(model.predict(X_test), axis=-1)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Test accuracy: {accuracy}")

    # Let's add the accuracy of the model to our evaluation report.
    evaluation_report = {
        "metrics": {
            "accuracy": {
                "value": accuracy
            },
        },
    }
    
    # We need to save the evaluation report to the output path.
    Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(Path(output_path) / "evaluation.json", "w") as f:
        f.write(json.dumps(evaluation_report))


if __name__ == "__main__":
    evaluate(
        model_path=MODEL_PATH, 
        test_path=TEST_PATH,
        output_path=OUTPUT_PATH
    )

Overwriting penguins/evaluation.py


## Step 2 - Testing the Evaluation Script

Let's test the script we just created by running it locally.

In [29]:
from penguins.preprocessor import preprocess
from penguins.train import train
from penguins.evaluation import evaluate


with tempfile.TemporaryDirectory() as directory:
    # First, we preprocess the data and create the 
    # dataset splits.
    preprocess(
        base_dir=directory, 
        data_filepath=DATA_FILEPATH
    )

    # Then, we train a model using the train and 
    # validation splits.
    train(
        base_directory=directory, 
        train_path=Path(directory) / "train", 
        validation_path=Path(directory) / "validation",
        epochs=10
    )
    
    # After training a model, we need to prepare a package just like
    # SageMaker would. This package is what the evaluation script is
    # expecting as an input.
    with tarfile.open(Path(directory) / "model.tar.gz", "w:gz") as tar:
        tar.add(Path(directory) / "model" / "001", arcname="001")
        
    
    # We can now call the evaluation script.
    evaluate(
        model_path=directory, 
        test_path=Path(directory) / "test",
        output_path=Path(directory) / "evaluation",
    )

Epoch 1/50
8/8 - 1s - loss: 1.1324 - accuracy: 0.1255 - val_loss: 1.1050 - val_accuracy: 0.1373
Epoch 2/50
8/8 - 0s - loss: 1.1005 - accuracy: 0.1799 - val_loss: 1.0847 - val_accuracy: 0.1765
Epoch 3/50
8/8 - 0s - loss: 1.0733 - accuracy: 0.2176 - val_loss: 1.0660 - val_accuracy: 0.2549
Epoch 4/50
8/8 - 0s - loss: 1.0483 - accuracy: 0.2678 - val_loss: 1.0497 - val_accuracy: 0.3922
Epoch 5/50
8/8 - 0s - loss: 1.0260 - accuracy: 0.3180 - val_loss: 1.0336 - val_accuracy: 0.4706
Epoch 6/50
8/8 - 0s - loss: 1.0045 - accuracy: 0.4770 - val_loss: 1.0183 - val_accuracy: 0.5294
Epoch 7/50
8/8 - 0s - loss: 0.9841 - accuracy: 0.6444 - val_loss: 1.0034 - val_accuracy: 0.6863
Epoch 8/50
8/8 - 0s - loss: 0.9638 - accuracy: 0.7490 - val_loss: 0.9886 - val_accuracy: 0.7451
Epoch 9/50
8/8 - 0s - loss: 0.9436 - accuracy: 0.7824 - val_loss: 0.9735 - val_accuracy: 0.7451
Epoch 10/50
8/8 - 0s - loss: 0.9231 - accuracy: 0.8033 - val_loss: 0.9580 - val_accuracy: 0.7451
Epoch 11/50
8/8 - 0s - loss: 0.9022 - a

INFO:tensorflow:Assets written to: /tmp/tmpp5y9747i/model/001/assets


Test accuracy: 0.6470588235294118


## Step 3 - Setting up a Processor

To run the evaluation script we can use a [Processing Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing). Check the [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) SageMaker's SDK documentation for more information.

This time, we will use a [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) running a TensorFlow image. This will give us access to every library we need to execute the evaluation script.

You can use the [sagemaker.image_utis.retrieve()](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html) function for generating the URI of pre-built docker images.

In [26]:
# Let's retrieve the image we want to use to run the
# processing job.
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4",
    py_version="py37",
    image_scope="training",
    instance_type="ml.m5.large"
)

# We can now setup the processor using the URI of
# the pre-built docker image.
evaluation_script_processor = ScriptProcessor(
    base_job_name="penguins-evaluation-processor",
    image_uri=image_uri,
    command=["python3"],
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
)

## Step 4 - Configuring the Model Input

One of the inputs to the Evaluation Step is the model we created. We explored two different ways to create the model: a Training Step and a Tuning Step.

Here we can configure the input to the Evaluation Step based on whether we want to select the best model generated by the Tuning Step, or the model we trained using the Training Step.

We can use the [get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model artifacts from the top performing training jobs of the hyperparameter tuning job.

In [27]:
# By default, this notebook uses the best model from the Tuning Step.
# You can set this variable to False if you want to use the result
# of the Training Step.
USE_TUNING_STEP = False

# This is the input in case we want to use the best model generated
# by the Tuning Step.
tuning_model_input = ProcessingInput(
    source=tuning_step.get_top_model_s3_uri(
        top_k=0, 
        s3_bucket=sagemaker_session.default_bucket()
    ),
    destination="/opt/ml/processing/model",
)

# This is the input in case we want to use the trained model
# from the Training Step.
training_model_input = ProcessingInput(
    source=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    destination="/opt/ml/processing/model"
)

# We can now select the appropriate input depending on which step
# we are using.
model_input = tuning_model_input if USE_TUNING_STEP else training_model_input

## Step 5 - Setting up a Processing Step

We can now create a [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) to run the evaluation script. We'll use the [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) we defined before. 

The inputs of this step will be the model and the test set that we generated during the preprocessing step. The output will be the evaluation report file.

The [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) lets us specify a list of [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile) instances from the output of the job. We can use this to map the evaluation report that we generate in the evaluations script. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information.

Also, we need to define the location where the evaluation step will be storing the evaluation report to prevent SageMaker from appending a timestamp to their auto-generated location. If we let SageMaker use a timestamp, we can't cache this step. That's the goal of the `evaluation_destination` parameter.

In [69]:
# This is the location where the evaluation step will store the 
# evaluation report.
evaluation_destination = ParameterString(
    name="evaluation_destination",
    default_value=f'{S3_FILEPATH}/evaluation',
)


# We want to map the evaluation report that we generate inside
# the evaluation script so we can later reference it.
evaluation_report = PropertyFile(
    name="evaluation-report",
    output_name="evaluation",
    path="evaluation.json"
)


# Notice how this step uses the model generated by the tuning or training
# step, and the test set generated by the preprocessing step.
evaluation_step = ProcessingStep(
    name="evaluation",
    processor=evaluation_script_processor,
    inputs=[
        model_input,
        ProcessingInput(
            source=preprocess_step.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=evaluation_destination),
    ],
    code="penguins/evaluation.py",
    property_files=[evaluation_report],
    cache_config=cache_config
)

## Step 6 - Running the Pipeline

We can now add the model evaluation step to the pipeline.

We are going to configure the pipeline to run the Tuning Step or the Training Step depending on the value of the `USE_TUNING_STEP` flag.

In [31]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        evaluation_destination,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step,
        evaluation_step
    ]
)

Submit the pipeline definition to the SageMaker Pipelines service to create a pipeline if it doesn't exist, or update the pipeline if it does.

In [32]:
if PIPELINE == "EVALUATION":
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()

## Step 7 - Cleaning up

Before you finish, don't forget to clean up after you.

In [ ]:
session3_pipeline.delete()

## Assignments

1. Extend the evaluation report by adding other metrics. For example, add the support of the test set (the number of samples.)

2. One of the assignments from the previous session was to replace the TensorFlow Estimator with a Pytorch Estimator. You can now modify the evaluation step to load a script that uses Pytorch to evaluate the model.

3. If you are runing the Training and Tuning Steps simultaneously, create two different Evaluation Steps to evaluate both models independently.

4. Instead of runing the Training and Tuning Steps simultaneously, run the Tuning Step but create two Evaluation Steps to evaluate the two best models produced by the Tuning Step. Check the [TuningJob.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to retrieve the two best models.

5. Modify the SageMaker Pipeline you created for the MNIST project and add an evaluation step. That step should use the test set you generated in the preprocessing step.

## Resources

1. Check the [ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) SageMaker's SDK documentation for more information about how to run a processing job using a machine learning framework.

2. SageMaker offers a list of pre-built docker images. You can use the [sagemaker.image_utis.retrieve()](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html) function for generating the URI of these images.

3. You can use the [TuningJob.get_top_model_s3_uri()](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.TuningStep.get_top_model_s3_uri) function to get the model artifacts from the top performing training jobs of the hyperparameter tuning job.

4. Check [How to Build and Manage Property Files](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-propertyfile.html) for more information about mapping the output of a [ProcessingStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.steps.ProcessingStep) to a [PropertyFile](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.properties.PropertyFile).

# Session 4 - Deploying the Model

This session extends the [SageMaker Pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines-sdk.html) with a step to register a new model if it reaches a predefined accuracy threshold.

In [33]:
from sagemaker import ModelPackage
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.predictor import Predictor
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join

## Step 1 - Creating Two New Pipeline Parameters

We are going to use two new [Pipeline Parameters](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-parameters.html) in our pipeline:

* `model_approval_status`: This parameter represents the default approval status that we will use when registering a new model. Check [Update the Approval Status of a Model](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-approve.html) for more information about the different approval status of a model and how you can update them.
* `accuracy_threshold`: This parameter represents the minimum accuracy that the model should reach in order for it to be registered.

In [34]:
model_approval_status = ParameterString(
    name="model_approval_status", 
    default_value="Approved"
)

accuracy_threshold = ParameterFloat(
    name="accuracy_threshold", 
    default_value=0.75
)

## Step 2 - Configuring the Model Assets

We need to specify the location of the model assets to register a model. We explored two different ways to create the model: a Training Step and a Tuning Step.

Here we can configure the model assets based on whether we want to select the best model generated by the Tuning Step, or the model we trained using the Training Step.

In [60]:
# This is the model data in case we want to use the best model generated
# by the Tuning Step.
tuning_model_data = tuning_step.get_top_model_s3_uri(
    top_k=0, 
    s3_bucket=sagemaker_session.default_bucket()
)

# This is the model data in case we want to use the trained model
# from the Training Step.
training_model_data = training_step.properties.ModelArtifacts.S3ModelArtifacts

# We can now select the appropriate model data depending on which step
# we are using.
model_data = tuning_model_data if USE_TUNING_STEP else training_model_data

## Step 3 - Configuring the Model

The model we trained uses TensorFlow, so we can use one the built-in TensorFlow inference image to create a [Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html).


In [36]:
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region=region,
    version="2.4",
    image_scope="inference",
    instance_type="ml.m5.large"
)

model = Model(
    image_uri=image_uri,
    model_data=model_data,
    sagemaker_session=PipelineSession(),
    role=role,
)

## Step 4 - Setting up the Model Metrics

When we register a model, we can specify a set of [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics). We can use the evaluation report we generated during the Model 
Evaluation step to populate these statistics.

In [ ]:
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(on="", values=[
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],
            "/evaluation.json"]
        ),
        content_type="application/json",
    )
)

## Step 5 - Setting up a Model Step

We can now create a [Model Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-model) to register the model. Check the [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) SageMaker's SDK documentation for more information. 

This step will use the Model we configured before.

In [37]:
model_package_group_name = "penguins-model-package-group"

register_model_step = ModelStep(
    name="register-model",
    step_args=model.register(
        content_types=["text/csv"],
        response_types=["text/csv"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.4",
        # sample_payload_url="",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_approval_status,
    ),
)

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:270: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Step 6 - Setting up a Condition Step

We only want to register a new model if its accuracy is above a predefined threshold. We can use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to accomplish this. Check the [ConditionStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#conditionstep) SageMaker's SDK documentation for more information.

In this example we are going to use a [ConditionGreaterThanOrEqualTo](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionGreaterThanOrEqualTo) condition to compare the model's accuracy with the threshold. Take a look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions.

In [38]:
# We can get the model accuracy directly from the evaluation
# report property file.
condition_gte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluation_step.name,
        property_file=evaluation_report,
        json_path="metrics.accuracy.value"
    ),
    right=accuracy_threshold
)

# If the condition succeeds, we can call the Model Step.
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[register_model_step],
    else_steps=[], 
)

## Step 7 - Running the Pipeline

We can now add the registration of the model to the pipeline. Notice how we add the condition step, which will call the Model Step if the condition passes.

In [39]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        evaluation_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step, 
        evaluation_step,
        condition_step
    ],
)

In [40]:
if PIPELINE == "DEPLOYMENT":
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()

## Step 8 - Loading the Latest Approved Model

Now that we registered the model, we can load the latest approved model from the Model Registry to deploy it to an endpoint.

We can use `boto3` to query the list of approved model packages and get the latest one that's been approved. Check the [boto3 SageMaker Client API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) for a list of every available method.

In [51]:
sagemaker_client = boto3.client("sagemaker")


def get_latest_approved_model_package(model_package_group_name):
    """
    Returns the latest approved model package registered under the 
    specified model package group.
    """
    try:
        # We can use the boto3 SageMaker's API to list the existing
        # model packages with the specified name. We only care about
        # approved models.
        response = sagemaker_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            ModelApprovalStatus="Approved",
            SortBy="CreationTime",
            MaxResults=100,
        )
        approved_packages = response["ModelPackageSummaryList"]

        # If we get a NextToken back, we need to deal with pagination.
        while len(approved_packages) == 0 and "NextToken" in response:
            response = sagemaker_client.list_model_packages(
                ModelPackageGroupName=model_package_group_name,
                ModelApprovalStatus="Approved",
                SortBy="CreationTime",
                MaxResults=100,
                NextToken=response["NextToken"],
            )
            approved_packages.extend(response["ModelPackageSummaryList"])

        if len(approved_packages) == 0:
            print(f"No approved model pacakages for \"{model_package_group_name}\"")
            return None

        # At this point we identified the latest approved model,
        # so we can return it.
        print(f"Latest approved model package: {approved_packages[0]['ModelPackageArn']}")
        return approved_packages[0]

    except ClientError as e:
        print(e.response["Error"]["Message"])
        raise Exception(e.response["Error"]["Message"])


We can now use the `get_latest_approved_model_package()` function to get the latest approved model from the Model Registry.

In [67]:
approved_model_package = get_latest_approved_model_package(model_package_group_name)
model_description = None

if approved_model_package:
    approved_model_package_arn = approved_model_package["ModelPackageArn"]

    model_description = sagemaker_client.describe_model_package(
        ModelPackageName=approved_model_package_arn
    )

model_description

Latest approved model package: arn:aws:sagemaker:us-east-1:325223348818:model-package/penguins-model-package-group/11


{'ModelPackageGroupName': 'penguins-model-package-group',
 'ModelPackageVersion': 11,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:325223348818:model-package/penguins-model-package-group/11',
 'CreationTime': datetime.datetime(2023, 4, 15, 12, 48, 23, 709000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins:latest',
    'ImageDigest': 'sha256:5cdafa465bf301b4b083eec1696ced447d5adc126a7bbc1afc0bf37011e07b1f',
    'ModelDataUrl': 's3://sagemaker-us-east-1-325223348818/penguins-2023-04-15-12-45-38-865/pipelines-vk9wv42955sy-register-custom-mode-p63e5Zz8WP/output/model.tar.gz',
    'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20',
     'SAGEMAKER_PROGRAM': 'inference.py',
     'SAGEMAKER_REGION': 'us-east-1',
     'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'},
    'Framework': 'TENSORFLOW',
    'FrameworkVersion': '2.4'}],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m5.large'],
  'SupportedCo

## Step 9 - Deploying the Model

We can now deploy the latest approved model to an endpoint if it doesn't already exist.

In [53]:
def does_endpoint_exist(endpoint_name):
    """
    Returns whether the supplied endpoint already exists.
    """
    try:
        sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
        return True
    except ClientError as e:
        return False

Using the arn of the model package from the Model Registry, we can deploy the model by creating a [ModelPackage](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.ModelPackage) instance and calling its `deploy()` method. The information of the model lives in the Model Registry so we don't need to specify anything else.

In [55]:
if PIPELINE == "DEPLOYMENT":
    model_package = ModelPackage(
        role=role, 
        model_package_arn=approved_model_package_arn, 
        sagemaker_session=sagemaker_session
    )

    endpoint_name = "penguins-endpoint"

    if not does_endpoint_exist(endpoint_name):
        model_package.deploy(
            endpoint_name=endpoint_name,
            initial_instance_count=1, 
            instance_type="ml.m5.large", 
        )

----!

## Step 10 - Testing the Endpoint

Using a [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor) from the endpoint name, we can test our model. Notice how the endpoint's interface expects the data to be transformed.

In [ ]:
if PIPELINE == "DEPLOYMENT":
    predictor = Predictor(endpoint_name=endpoint_name)

    # The payload we need to provide the model is in CSV format. Notice how the model expects data that's
    # already transformed. We can't provide the original data from our dataset because the model will not
    # work with it.
    payload = "0.6569590202313976, -1.0813829646495108, 1.2097102831892812, 0.9226343641317372, 1.0, 0.0, 0.0"
    p = predictor.predict(payload, initial_args={"ContentType": "text/csv"})

    # We can decode the output of the endpoint and print the "predictions" key.
    predictions = json.loads(p.decode("utf-8"))["predictions"]
    print(f"Prediction: {np.argmax(predictions, axis=1)[0]}")

## Step 11 - Cleaning up

Before you finish, don't forget to clean up after you.

In [ ]:
# Let's delete every model we registered under our model package group
for mp in sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
    print(f"Deleting {mp['ModelPackageArn']}")
    sagemaker_client.delete_model_package(ModelPackageName=mp["ModelPackageArn"])

# We can now delete the model package group.    
sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)

# And finally we delete the endpoint and the pipeline.
predictor.delete_endpoint()
pipeline.delete()

## Assignments

1. Modify your pipeline to add a new [Lambda Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-lambda) that's only called if the model's accuracy is not above the specified threshold. What you decide to do in the Lambda function is not important.

2. Modify your pipeline to add a new [Condition Step](#) that's called if the model's accuracy is not above the specified threshold. Set the condition to succeed if the accuracy is above 50%, in which case the model will be registered with a status of "PendingManualApproval." If the accuracy is not greater or equal to 50%, the model shouldn't be registered. In summary, the model should be registered with status "Approved" if its accuracy is greater or equal to 75% and with status "PendingManualApproval" if its accuracy is greater or equal to 50%.

3. Modify the payload that you send to the endpoint to classify multiple examples at once. Remember the payload is a CSV file, so you just need to add multiple lines to it.

4. Modify the SageMaker Pipeline you created for the MNIST project and add a step to register the model. Create two separate conditions using the metrics from your evaluation step to decide whether you should register the model. Both conditions have to be true to register the model.

## Resources

1. To learn more about the Model Registry, check [Register and Deploy Models with Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html).

2. Check [Update the Approval Status of a Model](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-approve.html) for more information about the different approval status of a model and how you can update them.

3. Check the [Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) SageMaker's SDK documentation for more information about the class uses by SageMaker to represent a model you can register and deploy.

4. Check the [ModelStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) SageMaker's SDK documentation. You can find an example of how to create a step to register a model in the [Model Step](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.model_step.ModelStep) page.

5. Check the [ModelMetrics](https://sagemaker.readthedocs.io/en/stable/api/inference/model_monitor.html#sagemaker.model_metrics.ModelMetrics) SageMaker's SDK documentation for more information about the metrics you can register with a model.

6. Check the [ConditionStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#conditionstep) SageMaker's SDK documentation. You can find an example of how to create a condition step the [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) page.

7. Take a look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions you can use in a pipeline.

8. Check the [boto3 SageMaker Client API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) for a list of every available method.

9. Check the [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor) SageMaker's SDK documentation for more information about connecting to an endpoint to run predictions.

# Session 5 - Custom Endpoints

Deploying a model directly to an endpoint has a problem: we don't have any control over the input and output of the model inside the endpoint. Fortunately, SageMaker gives us the ability to include an `inference.py` file with the model assets from where we can control how the endpoint works.

You can see more information about how this works by checking the [SageMaker TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container) documentation.

In the previous session, we deployed a model to an endpoint that expects the input formatted in CSV format. Instead, we want an endpoint that expects JSON. Since this is a production endpoint, we want to ensure it supports unprocessed data, just as customers will send it. Remember that the endpoint we deployed requires the input data to be transformed before it can process it.

Here is an example of the payload we want the endpoint to support:

```
{
    "island": "Biscoe",
    "culmen_length_mm": 48.6,
    "culmen_depth_mm": 16.0,
    "flipper_length_mm": 230.0,
    "body_mass_g": 5800.0,
}
```

In [259]:
import time

from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.lambda_helper import Lambda
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

## Step 1 - Preparing the Inference Code

Here is the inference code that we will include as part of the model assets to control the inference process on the SageMaker endpoint.

In [271]:
%%writefile container/code/inference.py

import os
import json
import requests
import numpy as np
import pandas as pd

from pickle import load
from pathlib import Path


MODEL_DIRECTORY = Path("/opt/ml/model/")


def handler(data, context, model_directory=MODEL_DIRECTORY):
    """
    This is the entrypoint that will be called by SageMaker when the endpoint
    receives a request.
    
    You can see more information at https://github.com/aws/sagemaker-tensorflow-serving-container.
    """
    print("Handling endpoint request")
    
    instance = _process_input(data, context, model_directory)
    output = _predict(instance, context)
    return _process_output(output, context)


def transform(payload, model_directory):
    print("Transforming input data...")
    pipeline = load(open(model_directory / "code" / "pipeline.pkl", 'rb'))
    
    island = payload.get("island", "Biscoe")
    culmen_length_mm = payload.get("culmen_length_mm", 0)
    culmen_depth_mm = payload.get("culmen_depth_mm", 0)
    flipper_length_mm = payload.get("flipper_length_mm", 0)
    body_mass_g = payload.get("body_mass_g", 0)
    
    data = pd.DataFrame(
        columns=["island", "culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"], 
        data=[[
            island, 
            culmen_length_mm, 
            culmen_depth_mm, 
            flipper_length_mm, 
            body_mass_g
        ]]
    )
    
    result = pipeline.transform(data)
    return result[0].tolist()
    

def _process_input(data, context, model_directory):
    print("Processing input data...")
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we can use the
        # data directly.
        endpoint_input = data
    elif context.request_content_type in ("application/json", "application/octet-stream"):
        # When the endpoint is running, we will receive a context
        # object. We need to parse the input and turn it into 
        # JSON in that case.
        endpoint_input = json.loads(data.read().decode("utf-8"))

        if endpoint_input is None:
            raise ValueError("There was an error parsing the input request.")
    else:
        raise ValueError(f"Unsupported content type: {context.request_content_type or 'unknown'}")
        
    return transform(endpoint_input, model_directory)


def _predict(instance, context):
    print("Sending input data to model to make a prediction...")
    
    model_input = json.dumps({"instances": [instance]})
    
    if context is None:
        # The context will be None when we are testing the code
        # directly from a notebook. In that case, we want to return
        # a fake prediction back.
        result = {
            "predictions": [
                [0.2, 0.5, 0.3]
            ]
        }
    else:
        # When the endpoint is running, we will receive a context
        # object. In that case we need to send the instance to the
        # model to get a prediction back.
        response = requests.post(context.rest_uri, data=model_input)
        
        if response.status_code != 200:
            raise ValueError(response.content.decode('utf-8'))
            
        result = json.loads(response.content)
    
    print(f"Response: {result}")
    return result


def _process_output(output, context):
    print("Processing prediction received from the model...")
    
    response_content_type = "application/json" if context is None else context.accept_header
    
    prediction = np.argmax(output["predictions"][0])
    confidence = output["predictions"][0][prediction]
    
    print(f"Prediction: {prediction}. Confidence: {confidence}")
    
    result = json.dumps({
        "prediction": str(prediction),
        "confidence": str(confidence)
    }), response_content_type
    
    return result

Overwriting container/code/inference.py


## Step 2 - Downloading the Scikit-Learn Pipeline

The inference code uses the Scikit-Learn preprocessing pipeline to transform the input data before sending it to the model. We need to make the pickled preprocessing pipeline available to the endpoint. Since we uploaded it to S3 as an output from the Preprocessing Step, we can now download it and store it locally.

In [272]:
# This is the location where we uploaded the pickled pipeline.
preprocessing_pipeline = f"{preprocessor_destination.default_value}/pipeline.pkl"

# Let's download the pipeline locally so we can test the inference code and 
# pack it with the model assets.
!aws s3 cp $preprocessing_pipeline container/code/pipeline.pkl

download: s3://mlschool/penguins/preprocessing/pipeline.pkl to container/code/pipeline.pkl


## Step 3 - Testing the Inference Code

Let's the test the inference code locally to make sure it works before deploying it. We can call the `handler()` function to test the inference code. This function is the entrypoint that will be called by SageMaker whenever the endpoint receives a request.

When we are testing the inference code we want to set the `context` to None so the function recognizes we are calling it locally. We also want to set the `model_directory` to the place where we saved the Scikit-Learn pipeline.

In [273]:
from container.code.inference import handler

handler(
    payload, 
    context=None, 
    model_directory=Path("container")
)

Handling endpoint request
Processing input data...
Transforming input data...
Sending input data to model to make a prediction...
Response: {'predictions': [[0.2, 0.5, 0.3]]}
Processing prediction received from the model...
Prediction: 1. Confidence: 0.5


/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
 

('{"prediction": "1", "confidence": "0.5"}', 'application/json')

## Step 4 - Creating a Custom Container

Our inference code uses TensorFlow to make predictions and Scikit-Learn to transform the raw data before sending it to the model. Unfortunately, the built-in TensorFlow SageMaker inference image doesn't come with Scikit-Learn preinstalled. We can solve this problem in two ways:

1. We could provide a `requirements.txt` file together with the `inference.py` file and SageMaker will install every library specified in that file.
2. We could create a custom container and use it to deploy our model. This option is much more involved but useful if you want complete control over the endpoint.

Creating a custom endpoint is much more fun, so let's do that.

To create a custom endpoint we need to define the container using a Dockerfile, build it, and publish it in the [Amazon Elastic Container Registry](https://aws.amazon.com/ecr/) (ECR). Unfortunately, SageMaker Studio notebooks can't run the docker command, so we need to create a separate SageMaker Notebook Instance and run the [container/container.ipynb](container/container.ipynb) notebook.

This is the Dockerfile that we'll use to create the custom image to deploy the model. Notice how this image inherits from a prebuilt TensorFlow Inference image.  

In [274]:
%%writefile container/Dockerfile

FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.4-cpu AS sagemaker

RUN apt-get clean && \
    apt-get update -y && \
    apt-get install -y --no-install-recommends \
    libgl1-mesa-glx

RUN pip install --upgrade pip
RUN pip install pandas
RUN pip install numpy
RUN pip install requests
RUN pip install tensorflow==2.4
RUN pip install scikit-learn==0.23.2

LABEL com.amazonaws.sagemaker.capabilities.multi-models=false
ENV SAGEMAKER_MULTI_MODEL=false

Overwriting container/Dockerfile


## Step 5 - Building the Custom Container

Create a SageMaker Notebook Instance, open the [container/container.ipynb](container/container.ipynb) notebook and run it. This notebook will store the custom image in the `/penguins:latest` ECR location.

In [275]:
caller_identify = !aws sts get-caller-identity --query 'Account' --output text
account_id = caller_identify[0]

CUSTOM_CONTAINER_IMAGE_URI = f"{account_id}.dkr.ecr.{region}.amazonaws.com/penguins:latest"
CUSTOM_CONTAINER_IMAGE_URI

'325223348818.dkr.ecr.us-east-1.amazonaws.com/penguins:latest'

## Step 6 - Repacking and Registering the Model

Let's register a new [Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) using our custom container. We also need to make sure SageMaker repackages the model assets to include the `inference.py` and `pipeline.pkl` files.

SageMaker triggers a repack when we specify the `Model.source_dir` attribute. We want that attribute to point to the local folder containing the files we want to include in the final package. SageMaker will automatically modify the original `model.tar.gz` package to include a `/code` folder containing the file in our local directory:

```
model/
    |--[model_version_number]
        |--assets/
        |--variables/
        |--saved_model.pb
code/
    |--inference.py
    |--pipeline.pkl
    |--requirements.txt
```

In [276]:
# Since we are specifying the `source_dir` attribute, SageMaker
# will trigger a repack. This will create a new step in the 
# pipeline right before registering the model.
custom_model = Model(
    image_uri=CUSTOM_CONTAINER_IMAGE_URI,
    model_data=model_data,
    entry_point="inference.py",
    source_dir="container/code",
    sagemaker_session=PipelineSession(),
    role=role,
)

register_custom_model_step = ModelStep(
    name="register-custom-model",
    step_args=custom_model.register(
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.4",
        # sample_payload_url="",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        approval_status=model_approval_status,
    )
)

/usr/local/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:270: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


## Step 7 - Preparing a Function to Deploy the Model

In the previous session we registered the model as part of the pipeline, but deployed it manually. Instead, we can use a [Lambda Step](#) to automatically deploy the model.

Let's start by writing the Lambda function that will take the model information and create a new endpoint hosting it.

In [277]:
%%writefile container/lambda.py

import os
import json
import boto3

sagemaker = boto3.client("sagemaker")

def lambda_handler(event, context):
    model_name = event["model_name"]
    endpoint_config_name = event["endpoint_config_name"]
    endpoint_name = event["endpoint_name"]
    role = event["role"]
    
    sagemaker.create_model(
        ModelName=model_name, 
        ExecutionRoleArn=role, 
        Containers=[{
            "ModelPackageName": event["model_package_arn"]
        }] 
    )

    sagemaker.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[
            {
                "ModelName": model_name,
                "InstanceType": "ml.m5.large",
                "InitialVariantWeight": 1,
                "InitialInstanceCount": 1,
                "VariantName": "AllTraffic",
            }
        ]
    )

    sagemaker.create_endpoint(
        EndpointName=endpoint_name, 
        EndpointConfigName=endpoint_config_name
    )
    
    return {
        "statusCode": 200,
        "body": json.dumps("Endpoint deployed successfully"),
        "model_name": model_name,
    }

Overwriting container/lambda.py


## Step 8 - Creating a Role for the Lambda Function

We need to create a new role to run the lambda function and give it access to SageMaker.

In [278]:
iam = boto3.client('iam')

def create_lambda_role(role_name):
    try:
        response = iam.create_role(
            RoleName = role_name,
            AssumeRolePolicyDocument = json.dumps({
                "Version": "2012-10-17",
                "Statement": [
                    {
                        "Effect": "Allow",
                        "Principal": {
                            "Service": "lambda.amazonaws.com"
                        },
                        "Action": "sts:AssumeRole"
                    }
                ]
            }),
            Description="Lambda Role"
        )

        role_arn = response['Role']['Arn']

        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )

        iam.attach_role_policy(
            PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
            RoleName=role_name
        )

        return role_arn

    except iam.exceptions.EntityAlreadyExistsException:
        response = iam.get_role(RoleName=role_name)
        return response['Role']['Arn']
    

lambda_role = create_lambda_role("lambda-deployment-role")

## Step 9 - Setting up the Lambda Step

Let's now define the [LambdaStep](#) that will run the function we defined before.

Notice how the Lambda Set uses the model information from the regisytration step.

In [279]:
# We can use the current time to generate a unique signature
# to generate new assets every time we deploy the model.
signature = time.strftime("%m%d%H%M%S", time.localtime())
model_name = "penguins-model-" + signature
endpoint_config_name = "penguins-endpoint-config-" + signature
endpoint_name = "penguins-endpoint-" + signature
lambda_function_name = "deploy-endpoint-fn-" + signature 


deploy_step = LambdaStep(
    name="deploy",
    lambda_func=Lambda(
        function_name=lambda_function_name,
        execution_role_arn=lambda_role,
        script="container/lambda.py",
        handler="lambda.lambda_handler",
        timeout=600,
        memory_size=10240,
    ),
    inputs={
        "model_name": model_name,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
        "model_package_arn": register_custom_model_step.properties.ModelPackageArn,
        "role": role,
    },
    outputs=[
        LambdaOutput(output_name="model_name", output_type=LambdaOutputTypeEnum.String)
    ]
)

## Step 10 - Setting up the Condition Step

We only want to register a new model if its accuracy is above a predefined threshold. We can use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to accomplish this. Check the [ConditionStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#conditionstep) SageMaker's SDK documentation for more information.

In this example we are going to use a [ConditionGreaterThanOrEqualTo](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionGreaterThanOrEqualTo) condition to compare the model's accuracy with the threshold. Take a look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions.

If the condition succeeds, we will register the model and deploy it.

In [280]:
# If the condition succeeds, we can call the Model Step.
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[
        register_custom_model_step, deploy_step
    ],
    else_steps=[], 
)

## Step 11 - Running the Pipeline

We can now use the new Condition Step and run the pipeline.

In [281]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        evaluation_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        tuning_step if USE_TUNING_STEP else training_step, 
        evaluation_step,
        condition_step
    ],
)

In [282]:
if PIPELINE == "CUSTOM_ENDPOINT":
    pipeline.upsert(role_arn=role)
    execution = pipeline.start()

Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.


## Step 12 - Testing the Endpoint

We can now create a [Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html) to test the endpoint. Notice how you can specify a serializer and a deserializer to have the predictor automatically serialize and deserialize the information to and from the endpoint.

Check [Serializers](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html) and [Deserializers](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html) for a list of supported serializers and deserializers.

In [283]:
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

Running one example through the endpoint.

In [265]:
predictor.predict({
    "island": "Dream",
    "culmen_length_mm": 46.4,
    "culmen_depth_mm": 18.6,
    "flipper_length_mm": 190.0,
    "body_mass_g": 3450.0,
    
})

{'prediction': '0', 'confidence': '0.624125779'}

Running another example.

In [266]:
predictor.predict({
    "island": "Biscoe",
    "culmen_length_mm": 48.6,
    "culmen_depth_mm": 16.0,
    "flipper_length_mm": 230.0,
    "body_mass_g": 5800.0,
})

{'prediction': '2', 'confidence': '0.993786812'}

## Step 13 - Cleaning up

Before you finish, don't forget to clean up after you.

In [288]:
# # Let's delete every model we registered under our model package group
# for mp in sagemaker_client.list_model_packages(ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"]:
#     print(f"Deleting {mp['ModelPackageArn']}")
#     sagemaker_client.delete_model_package(ModelPackageName=mp["ModelPackageArn"])

# # We can now delete the model package group.    
# sagemaker_client.delete_model_package_group(ModelPackageGroupName=model_package_group_name)


# predictor.delete_endpoint()


# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sagemaker_client.delete_model(ModelName=model_name)

# pipeline.delete()

{'ResponseMetadata': {'RequestId': '98e5442c-41f2-4fb2-b8ff-59fde66e64d4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '98e5442c-41f2-4fb2-b8ff-59fde66e64d4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 15 Apr 2023 17:19:37 GMT'},
  'RetryAttempts': 0}}

## Assignments

## Resources


[SageMaker TensorFlow Serving Container](https://github.com/aws/sagemaker-tensorflow-serving-container)

# Session 6 - Model Monitoring

In [337]:
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import DataQualityCheckConfig, QualityCheckStep, ModelQualityCheckConfig
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.model_monitor import DatasetFormat, model_monitoring
from sagemaker.drift_check_baselines import DriftCheckBaselines
from sagemaker.workflow.parameters import ParameterBoolean

## Step 1 - Data Quality Configuration

In [330]:
data_quality_skip_check = ParameterBoolean(
    name="data_quality_skip_check", 
    default_value=True
)

data_quality_register_new_baseline = ParameterBoolean(
    name="data_quality_register_new_baseline", 
    default_value=True
)

data_quality_supplied_baseline_statistics = ParameterString(
    name="data_quality_supplied_baseline_statistics", 
    default_value=""
)

data_quality_supplied_baseline_constraints = ParameterString(
    name="data_quality_supplied_baseline_constraints", 
    default_value=""
)


## Step 2 - Setting up Data Quality Check Step

In [331]:
check_job_config = CheckJobConfig(
    instance_type="ml.c5.xlarge",
    instance_count=1,
    sagemaker_session=sagemaker_session,
    volume_size_in_gb=120,
    role=role,
)

data_quality_check_config = DataQualityCheckConfig(
    baseline_dataset=preprocess_step.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
    dataset_format=DatasetFormat.csv(header=False, output_columns_position="START"),
    output_s3_uri=Join(on='/', values=['s3:/', BUCKET, "monitor", "dataqualitycheckstep"]),
)

data_quality_check_step = QualityCheckStep(
    name="data-quality-check",
    skip_check=data_quality_skip_check,
    register_new_baseline=data_quality_register_new_baseline,
    quality_check_config=data_quality_check_config,
    check_job_config=check_job_config,
    supplied_baseline_statistics=data_quality_supplied_baseline_statistics,
    supplied_baseline_constraints=data_quality_supplied_baseline_constraints,
    model_package_group_name=model_package_group_name,
)


## Step 3 - Model Quality Configuration

In [332]:
model_quality_skip_check = ParameterBoolean(
    name="model_quality_skip_check", 
    default_value=True
)

model_quality_register_new_baseline = ParameterBoolean(
    name="model_quality_register_new_baseline", 
    default_value=True
)

model_quality_supplied_baseline_statistics = ParameterString(
    name="model_quality_supplied_baseline_statistics", 
    default_value=""
)

model_quality_supplied_baseline_constraints = ParameterString(
    name="model_quality_supplied_baseline_constraints", 
    default_value=""
)

## Step 4 - Setting up Model Quality Check Step

In [334]:
model_quality_check_config = ModelQualityCheckConfig(
    baseline_dataset=model_data,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=Join(on='/', values=['s3:/', BUCKET, "monitor", "modelqualitycheckstep"]),
    problem_type="Classification",
    inference_attribute="_c0", # use auto-populated headers since we don't have headers in the dataset
    ground_truth_attribute="_c1", # use auto-populated headers since we don't have headers in the dataset
)

model_quality_check_step = QualityCheckStep(
    name="model-quality-check",
    skip_check=model_quality_skip_check,
    register_new_baseline=model_quality_register_new_baseline,
    quality_check_config=model_quality_check_config,
    check_job_config=check_job_config,
    supplied_baseline_statistics=model_quality_supplied_baseline_statistics,
    supplied_baseline_constraints=model_quality_supplied_baseline_constraints,
    model_package_group_name=model_package_group_name,
)

## Step 5 - Setting up the Model Metrics

In [335]:
model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_check_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_check_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=Join(on="", values=[
            evaluation_step.arguments['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri'],
            "/evaluation.json"]
        ),
        content_type="application/json",
    ),
    
    # model_statistics=MetricsSource(
    #     s3_uri=model_quality_check_step.properties.CalculatedBaselineStatistics,
    #     content_type="application/json",
    # ),
    
    model_constraints=MetricsSource(
        s3_uri=model_quality_check_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

## Step 6 - Setting up the Drift Check Baselines

In [338]:
drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_check_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_check_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_check_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_check_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

## Step 7 - Setting up a Model Step

In [339]:
register_model_step = ModelStep(
    name="register-model",
    step_args=custom_model.register(
        content_types=["application/json"],
        response_types=["application/json"],
        inference_instances=["ml.m5.large"],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="TENSORFLOW",
        framework_version="2.4",
        model_package_group_name=model_package_group_name,
        model_metrics=model_metrics,
        drift_check_baselines=drift_check_baselines,
        approval_status=model_approval_status,
    )
)

## Step 8 - Setting up the Condition Step

We only want to register a new model if its accuracy is above a predefined threshold. We can use a [Condition Step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-condition) to accomplish this. Check the [ConditionStep](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#conditionstep) SageMaker's SDK documentation for more information.

In this example we are going to use a [ConditionGreaterThanOrEqualTo](https://sagemaker.readthedocs.io/en/stable/workflows/pipelines/sagemaker.workflow.pipelines.html#sagemaker.workflow.conditions.ConditionGreaterThanOrEqualTo) condition to compare the model's accuracy with the threshold. Take a look at the [Conditions](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_building_pipeline.html#conditions) section in the documentation for more information about the types of supported conditions.

If the condition succeeds, we will register the model and deploy it.

In [340]:
condition_step = ConditionStep(
    name="check-model-accuracy",
    conditions=[condition_gte],
    if_steps=[
        register_model_step, deploy_step
    ],
    else_steps=[], 
)

## Step 9 - Running the Pipeline

In [341]:
pipeline = Pipeline(
    name="penguins-pipeline",
    parameters=[
        dataset_location, 
        preprocessor_destination,
        data_quality_skip_check,
        data_quality_register_new_baseline,
        data_quality_supplied_baseline_statistics,
        data_quality_supplied_baseline_constraints,
        model_quality_skip_check,
        model_quality_register_new_baseline,
        model_quality_supplied_baseline_statistics,
        model_quality_supplied_baseline_constraints,
        evaluation_destination,
        model_approval_status,
        accuracy_threshold,
    ],
    steps=[
        preprocess_step, 
        data_quality_check_step,
        tuning_step if USE_TUNING_STEP else training_step, 
        model_quality_check_step,
        evaluation_step,
        condition_step
    ],
)

In [342]:
pipeline.upsert(role_arn=role)
execution = pipeline.start()

Popping out 'CertifyForMarketplace' from the pipeline definition since it will be overridden in pipeline execution time.


## Assignments

## Resources

# Notes

* [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit)
* [Amazon SageMaker Model Monitor](https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html)